In [8]:
import environ
from openai import OpenAI
import openai
# from pathlib import Path

# Import OpenAI key
env = environ.Env()
environ.Env.read_env()
API_KEY = env("OPENAI_API_KEY")
openai.api_key = API_KEY

# Set constanst
LLM_MODEL = "gpt-3.5-turbo"
TEMPERATURE = 0

In [9]:
system_content = f"""
You are an expert non-violent comunication.
Your task is to identify toxic language.
Always answer using three labels:
 - 'neutral': use this category for neutral non-toxic language.
 - 'toxic': use this category for toxic or violent language.
 - 'unclear': use this category is you don't know what category select.
You must provide me with a label that classify the sentence provided below.
"""

In [10]:
client_openai = OpenAI()

In [11]:
completion = client_openai.chat.completions.create(
  model=LLM_MODEL,
  messages=[
    {"role": "system", "content": system_content},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ],
  temperature=TEMPERATURE,
)

print(completion.choices[0].message)
print(completion.usage)

ChatCompletionMessage(content='neutral', role='assistant', function_call=None, tool_calls=None)
CompletionUsage(completion_tokens=1, prompt_tokens=107, total_tokens=108)


In [12]:
def clf_sentence(client_openai, system_content, user_content, LLM_MODEL, TEMPERATURE):
    completion = client_openai.chat.completions.create(
        model=LLM_MODEL,
        messages=[
            {"role": "system", "content": system_content},
            {"role": "user", "content": user_content}
        ],
        temperature=TEMPERATURE,
    )

    selected_label = completion.choices[0].message.content
    used_tokens = {
        "completion_tokens":completion.usage.completion_tokens,
        "prompt_tokens":completion.usage.prompt_tokens,
        "total_tokens":completion.usage.total_tokens
    }

    return selected_label, used_tokens

In [13]:
label, token_dict = clf_sentence(client_openai, system_content, "what the fuck are you talking about?", LLM_MODEL, TEMPERATURE)

print(label)
print(token_dict)

toxic
{'completion_tokens': 2, 'prompt_tokens': 103, 'total_tokens': 105}


In [14]:
toxic_sentences = [
    # Toxic sentences
    "You don't deserve me.",
    "You're pathetic.",
    "I hate you.",
    "You're a bad parent.",
    "You're being crazy.",
    "You're so needy",
    # Neutral sentences
    "Oh! That reminds me I have to put my doctor's appointment in my calendar.",
    "Stop asking if I'm okay. Everything is fine",
    "These are not the droids you are looking for"
]

In [15]:
label_list = []

for toxic_sentence in toxic_sentences:
    # print(toxic_sentence)
    label, _ = clf_sentence(
        client_openai,
        system_content,
        toxic_sentence,
        LLM_MODEL,
        TEMPERATURE)
    # print(label)
    label_list.append((toxic_sentence, label))

In [16]:
label_list

[("You don't deserve me.", 'toxic'),
 ("You're pathetic.", 'toxic'),
 ('I hate you.', 'toxic'),
 ("You're a bad parent.", 'toxic'),
 ("You're being crazy.", 'toxic'),
 ("You're so needy", 'toxic'),
 ("Oh! That reminds me I have to put my doctor's appointment in my calendar.",
  'neutral'),
 ("Stop asking if I'm okay. Everything is fine", 'neutral'),
 ('These are not the droids you are looking for', 'neutral')]

In [19]:
a = 11
a += 2
a

13